In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

Using TensorFlow backend.


In [2]:
innInfo = pd.read_csv('https://www.dropbox.com/s/5aztgcxa5euubn4/inn_info_public.csv?dl=1')
pays = pd.read_csv('https://www.dropbox.com/s/7if97r10dc2i441/pays.csv?dl=1')

In [3]:
innInfo = innInfo.iloc[:, 0:3]

In [5]:
part1 = pd.merge(innInfo, pays,
                 how='inner', left_on='hash_inn', right_on='hash_inn_kt', sort=True)
# удалим старый hash_inn_kt
part1.drop('hash_inn_kt', axis=1, inplace=True)
# и переименуем hash_inn_dt
part1.rename(columns={'hash_inn_dt': 'another_inn'}, inplace=True)
# добавим kt столбец с true значениями
part1.loc[:,'kt'] = pd.Series(np.ones(part1.shape[0], dtype='bool'), index=part1.index)

part2 = pd.merge(innInfo, pays,
                 how='inner', left_on='hash_inn', right_on='hash_inn_dt', sort=True)
# удалим старый hash_inn_dt
part2.drop('hash_inn_dt', axis=1, inplace=True)
# и переименуем hash_inn_kt
part2.rename(columns={'hash_inn_kt': 'another_inn'}, inplace=True)
# добавим kt столбец с true значениями
part2.loc[:,'kt'] = pd.Series(np.zeros(part2.shape[0], dtype='bool'), index=part2.index)

MemoryError: 

In [ ]:
# объединим датасеты
df = pd.concat((part1, part2))

In [ ]:
cols = list(df)
cols[1], cols[7] = cols[7], cols[1]
df = df[cols]

In [ ]:
df = df[df['okved2'] != -1]
df.dropna(inplace=True)

In [ ]:
work_df = df.iloc[np.random.choice(len(df), 500000),:]
X_train_work, X_test_work, y_train_work, y_test_work = train_test_split(work_df.iloc[:, 0:7],
                                                work_df['okved2'], test_size=0.2, random_state=17)

# Keras

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(10, input_shape=(7, ), activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(80, activation='softmax'))
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_work, y_train_work, batch_size=32, epochs=10, validation_data=(X_test_work, y_test_work))

In [ ]:
model.summary()